# Mount to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Improt required libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import scipy.io as sio
import numpy as np
import pickle as pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn import metrics
from scipy import signal
import os
import math

import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
# from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
import warnings
warnings.filterwarnings('ignore')

# Data Pre-processing

In [ ]:
dataset_path = '/content/drive/My Drive/Major/BED_dataset_RAW_PARSED/'
channel_names = ['P4', 'FC6', 'F8', 'T7']

In [ ]:
def load_eeg_column_data(directory, col_index):
    data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith('.mat'):
            filepath = os.path.join(directory, filename)

            mat_contents = sio.loadmat(filepath)
            recordings = mat_contents['recording']

            recordings = recordings[:30208, col_index]
            recordings = recordings.T

            data.append(recordings)

            # Extract label from filename
            start_index = filename.index('s') + 1
            end_index = filename.index('_')
            # Extract the substring between 's' and '_'
            number = filename[start_index:end_index]
            person_id = int(number) # Convert the extracted number to an integer

            label = person_id - 1
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

In [ ]:
eeg_Data_P4, labels = load_eeg_column_data(dataset_path, 10)
eeg_Data_FC6, _ = load_eeg_column_data(dataset_path, 14)
eeg_Data_F8, _ = load_eeg_column_data(dataset_path, 12)
eeg_Data_T7, _ = load_eeg_column_data(dataset_path, 6)

In [ ]:
# Change the label from subject_id to categorial label
Y = to_categorical(labels)

(63, 21)

# Feature Extraction

In [ ]:
# Spectogram
def extract_features(eeg_Data):
    fs = 200

    size_dataset = len(eeg_Data)
    nperseg = 400
    noverlap = 200
    f_size = math.ceil(((nperseg + 1)/2))
    t_size = int((eeg_Data[0].size - noverlap ) / (nperseg - noverlap))
    X_full = np.ndarray(shape=(size_dataset, f_size, t_size, 3))

    for i in range(0, size_dataset):
      for j in range(0,3):
        X = eeg_Data[i]
        f, t, Sxx = spectrogram(X, fs, nperseg = nperseg, noverlap = noverlap, mode='complex')
        X_full[i, :, :, j] = Sxx

    X_full = np.array(X_full)
    return X_full

In [ ]:
X_full_P4 = extract_features(eeg_Data_P4)
X_full_FC6 = extract_features(eeg_Data_FC6)
X_full_F8 = extract_features(eeg_Data_F8)
X_full_T7 = extract_features(eeg_Data_T7)


In [ ]:
X_full_P4 /= 255
X_full_FC6 /= 255
X_full_F8 /= 255
X_full_T7 /= 255

# Load Models

In [ ]:
models_path = '/content/drive/My Drive/saved_models'

model_files = [filename for filename in os.listdir(models_path) if filename.endswith('.h5')]

loaded_models = {}

for filename in model_files:
    # Extract channel name from filename
    channel = filename.split('_')[1].split('.h5')[0]
    model_path = os.path.join(models_path, filename)
    loaded_models[channel] = load_model(model_path)

print("Models loaded!!")
# Now loaded_models is a dictionary with keys as channel names and values as loaded models
print(loaded_models)

Models loaded!!
{'P4': <keras.src.engine.sequential.Sequential object at 0x7df3f84a6500>, 'F8': <keras.src.engine.sequential.Sequential object at 0x7df3b4771660>, 'T7': <keras.src.engine.sequential.Sequential object at 0x7df3f84a7340>, 'FC6': <keras.src.engine.sequential.Sequential object at 0x7df3b4663460>}


# Get predictions

In [ ]:
pred_Y_P4 = loaded_models["P4"].predict(X_full_P4)
pred_Y_P4=np.argmax(pred_Y_P4,axis=1)

pred_Y_FC6 = loaded_models["FC6"].predict(X_full_FC6)
pred_Y_FC6=np.argmax(pred_Y_FC6,axis=1)

pred_Y_F8 = loaded_models["F8"].predict(X_full_F8)
pred_Y_F8=np.argmax(pred_Y_F8,axis=1)

pred_Y_T7 = loaded_models["T7"].predict(X_full_T7)
pred_Y_T7=np.argmax(pred_Y_T7,axis=1)

8/8 [==============================] - 18s 2s/step


In [ ]:
# convert categorial labels to initial label
label_Y = np.argmax(Y, axis=1)

# Voting

In [ ]:
import numpy as np

def hard_voting(predictions):
    # Calculate the majority class label
    majority_label = np.bincount(predictions).argmax()

    # Count the occurrences of the majority label
    majority_count = np.count_nonzero(predictions == majority_label)

    # Check if the majority label appears in at least 75% of the predictions
    if majority_count >= 0.75 * len(predictions):
        # Return the majority label as the final prediction
        return majority_label
    else:
        # Return -1 if 75% of the elements are not the same
        return -1


In [ ]:
voted_Y = []
for i in range(len(label_Y)):
    result = np.array([pred_Y_P4[i],pred_Y_FC6[i],pred_Y_F8[i],pred_Y_T7[i]])
    pred = hard_voting(result)
    voted_Y.append(pred)

voted_Y = np.array(voted_Y)

# Plot metrics

In [ ]:
print(metrics.classification_report(label_Y,voted_Y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.88      1.00      0.93        21
           2       1.00      0.95      0.97        19
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        21
           5       1.00      0.90      0.95        21
           6       1.00      1.00      1.00        15
           7       1.00      0.83      0.91        12
           8       1.00      0.89      0.94        18
           9       1.00      0.90      0.95        20
          10       0.92      1.00      0.96        23
          11       1.00      1.00      1.00        15
          12       0.82      1.00      0.90        14
          13       1.00      0.95      0.97        20
          14       0.89      0.94      0.92        18
          15       1.00      1.00      1.00        16
          16       1.00      1.00      1.00        13
          17       1.00    